In [1]:
import tensorflow as tf
tf.__version__

'2.5.0'

In [2]:
import pandas as pd

In [3]:
import numpy as np

In [4]:
import torch
torch.__version__

'1.6.0+cpu'

In [5]:
#!pip install pandas

In [6]:
#!pip install tensorflow

In [7]:
#!pip install transformers

In [8]:
#!pip install torch==1.6.0+cpu torchvision==0.7.0+cpu -f https://download.pytorch.org/whl/torch_stable.html

In [9]:
#!pip install tf-nightly

In [10]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [11]:
#tokens, token_embs, subtokens, subtoken_embs, sent_max_embs, sent_mean_embs, bert_pooler_outputs = m(texts)

In [12]:
from transformers import AutoTokenizer, TFAutoModel

In [13]:
tokenizer_bert = AutoTokenizer.from_pretrained('Geotrend/bert-base-ru-cased')

In [14]:
model_bert = TFAutoModel.from_pretrained('Geotrend/bert-base-ru-cased')

Some layers from the model checkpoint at Geotrend/bert-base-ru-cased were not used when initializing TFBertModel: ['mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at Geotrend/bert-base-ru-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [15]:
texts = ['проводите ли вы обучения по работе со сметными программами']

In [16]:
input_ids = tokenizer_bert(texts,
                     max_length=20, truncation=True, padding='max_length', return_token_type_ids=False, return_tensors='tf')
input_ids

{'input_ids': <tf.Tensor: shape=(1, 20), dtype=int32, numpy=
array([[   11,  1436,  9793,   300,  5058, 12777,  8827,   369,  5292,
          710,  2396,  1029,  1904,  9492,   494,    12,     0,     0,
            0,     0]])>, 'attention_mask': <tf.Tensor: shape=(1, 20), dtype=int32, numpy=array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0]])>}

In [17]:
outputs = model_bert(input_ids.input_ids)

In [18]:
outputs[0].shape

TensorShape([1, 20, 768])

In [19]:
#outputs[0][0] - наш эмбеддинг

In [20]:
df = pd.read_excel('data/QAfinal.xls')

In [21]:
#df['Question'] = df['Question'].apply(lambda x: x + "?")

In [22]:
df.head(2)

,Question,Answer,Class,Recommend
0,Сколько стоит Грандсмета?,"Стоимость ПК ""Грандсмета"" 29 000 рублей.",0,Мне нужно обучение* Сколько стоит ТЕР
1,Продаете ли вы Грандсмета?,"Стоимость ПК ""Грандсмета"" 29 000 рублей.",0,Мне нужно обучение* Сколько стоит ТЕР


In [23]:
input_ids

{'input_ids': <tf.Tensor: shape=(1, 20), dtype=int32, numpy=
array([[   11,  1436,  9793,   300,  5058, 12777,  8827,   369,  5292,
          710,  2396,  1029,  1904,  9492,   494,    12,     0,     0,
            0,     0]])>, 'attention_mask': <tf.Tensor: shape=(1, 20), dtype=int32, numpy=array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0]])>}

In [24]:
input_ids = tokenizer_bert(list(df['Question'].values),
                max_length=20, truncation=True, padding='max_length', return_token_type_ids=False, return_tensors='tf')

In [25]:
outputs = model_bert(input_ids)

In [26]:
outputs.pooler_output

<tf.Tensor: shape=(240, 768), dtype=float32, numpy=
array([[-0.2988714 , -0.07024034,  0.03226464, ...,  0.25998107,
         0.13052884, -0.2484819 ],
       [-0.29257223,  0.14096244, -0.05034341, ...,  0.2808302 ,
         0.11817037, -0.15142749],
       [-0.4058805 ,  0.12957962,  0.02636607, ...,  0.30060488,
         0.16971219, -0.18909742],
       ...,
       [-0.22889788,  0.16740955, -0.10010586, ...,  0.42261437,
         0.06502905, -0.25097334],
       [-0.28832248,  0.16049331, -0.08975539, ...,  0.38184303,
         0.20787871, -0.2903126 ],
       [-0.23420304, -0.02518237, -0.11990237, ...,  0.28935155,
        -0.05763339,  0.01517892]], dtype=float32)>

In [27]:
x_train = outputs.pooler_output

In [28]:
y_train = tf.keras.utils.to_categorical(df['Class'].values, num_classes=None, dtype='float32')

In [29]:
x_train.shape, y_train.shape

(TensorShape([240, 768]), (240, 12))

In [30]:
fc_model = Sequential([
    tf.keras.Input(shape=(768,)),
    Dense(256, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    Dense(128, activation='relu'),
    Dense(12, activation='softmax')
])

fc_model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

fc_model.fit(x_train, y_train, epochs=35, verbose=1)

fc_model.save('models/classificator.h5')

Epoch 1/35
8/8 [==============================] - 0s 3ms/step - loss: 2.4732 - accuracy: 0.1292
Epoch 2/35
8/8 [==============================] - 0s 2ms/step - loss: 2.2817 - accuracy: 0.1875
Epoch 3/35
8/8 [==============================] - 0s 3ms/step - loss: 2.1036 - accuracy: 0.3458
Epoch 4/35
8/8 [==============================] - 0s 2ms/step - loss: 1.9590 - accuracy: 0.4042
Epoch 5/35
8/8 [==============================] - 0s 2ms/step - loss: 1.8347 - accuracy: 0.4208
Epoch 6/35
8/8 [==============================] - 0s 3ms/step - loss: 1.7029 - accuracy: 0.4458
Epoch 7/35
8/8 [==============================] - 0s 2ms/step - loss: 1.5133 - accuracy: 0.5667
Epoch 8/35
8/8 [==============================] - 0s 2ms/step - loss: 1.4358 - accuracy: 0.5625
Epoch 9/35
8/8 [==============================] - 0s 2ms/step - loss: 1.2777 - accuracy: 0.6125
Epoch 10/35
8/8 [==============================] - 0s 2ms/step - loss: 1.2009 - accuracy: 0.6250
Epoch 11/35
8/8 [======================

In [36]:
test_model = tf.keras.models.load_model('models/classificator.h5')

In [37]:
from dicts import terms_dict 
import re

In [38]:
def sub_by_dict(text, dictionary):
    for word in text.split():
        cleared_word = re.sub('[.!?&,]', '', word) # за некоторомы словами следуют знаки препинания
        if cleared_word in dictionary.keys():
            text = text.replace(cleared_word, dictionary[cleared_word])
    return text

In [39]:
def get_qa_answer(text):
    text = sub_by_dict(text, terms_dict)
    question = tokenizer_bert(text,
                max_length=20, truncation=True, padding='max_length', return_token_type_ids=False, return_tensors='tf')
    question_out =  model_bert(question)
    question_emb = question_out.pooler_output
    pred = test_model.predict(question_emb)
    score = (pred[0][np.argmax(pred)])

    answer = df.loc[df.Class == np.argmax(pred)]['Answer'].values[0]
    print(text)
    return answer, score

In [54]:
testing = [
    'Проводите ли вы обучение?',
    'вы обучаете работать с грандсмета',
    "Есть ли у вас обучение по работе?",
    "я хочу записаться на обучение по А-ноль",
    "сколько стоят фер для А-ноль?",
    "сколько стоят фер для А-ноль",
    "можно ли купить А-ноль и фер?",
    "сколько стоит федералка для А-ноль",
    "ты плохой человек",
    "Я тебя ненавижу, мудак"    
]

In [55]:
for question in testing:
    anw, scr = get_qa_answer(question)
    print(anw, scr, '\n')

Проводите ли вы обучение?
РЦЦС проводит однодневные курсы по работе с А-ноль и Грандсметой. Стоимость - 4 200 рублей за человека. Даты проведения определяются по мере набора групп. 0.8824413 

вы обучаете работать с грандсмета
РЦЦС проводит однодневные курсы по работе с А-ноль и Грандсметой. Стоимость - 4 200 рублей за человека. Даты проведения определяются по мере набора групп. 0.93944144 

Есть ли у вас обучение по работе?
РЦЦС проводит однодневные курсы по работе с А-ноль и Грандсметой. Стоимость - 4 200 рублей за человека. Даты проведения определяются по мере набора групп. 0.8989382 

я хочу записаться на обучение по А-ноль
РЦЦС проводит однодневные курсы по работе с А-ноль и Грандсметой. Стоимость - 4 200 рублей за человека. Даты проведения определяются по мере набора групп. 0.8693889 

сколько стоят  федеральные единичные расценки для А-ноль?
Цена на ПК "А-ноль" составляет 43 000 рублей.  Стоимость базы ФЕР для ПК "А-ноль" 10 000 рублей, годовая подписка - 10 000 рублей. 0.888912